## Problem 3: How many people live in the dominance area of each shopping center? 

Find out how many people live under the dominance area of each shopping center. You should be able to do this based on the output from problem 2, lesson materials from weeks 2, 3 and 4 and these additional hints:

- You can read in the population data from the HSY WFS service: https://kartta.hsy.fi/geoserver/wfs (check more hints from the lesson materials)
- Aggregate your dominance areas from problem 2 into a unified geometries using [`dissolve()`](http://geopandas.org/aggregation_with_dissolve.html#dissolve-example) -function in geopandas before joining with the population data.
- Remember to check the crs of the input data.
- Join information between the population grid and the dominance areas -layer using `intersect` as the condition in the spatial join.

You can freely organize your code into the code cells below.



In [1]:
# Import modules
import requests
import geopandas as gpd
from pyproj import CRS
import geojson

In [2]:
# Specify the url for web feature service
url = "https://kartta.hsy.fi/geoserver/wfs"

# Specify parameters (read data in json format)
params = dict(
    service="WFS",
    version="2.0.0",
    request="GetFeature",
    typeName="asuminen_ja_maankaytto:Vaestotietoruudukko_2018",
    outputFormat="json"
)

# Fetch data from WFS using requests
resp = requests.get(url, params=params)

# Create GeoDataFrame from geojson
pop = gpd.GeoDataFrame.from_features(geojson.loads(resp.content))

# Change the name of a column
pop = pop.rename(columns={"asukkaita": "pop18"})

# Subset columns
pop = pop[["pop18", "geometry"]]

# Define CRS in case there is none
if not pop.crs:
    pop.crs = CRS.from_epsg(3879).to_wkt()
    
pop.head()

,pop18,geometry
0,9,"POLYGON ((25472499.995 6689749.005, 25472499.9..."
1,5,"POLYGON ((25472499.995 6685998.998, 25472499.9..."
2,8,"POLYGON ((25472499.995 6684249.004, 25472499.9..."
3,5,"POLYGON ((25472499.995 6683999.005, 25472499.9..."
4,11,"POLYGON ((25472499.995 6682998.998, 25472499.9..."


In [3]:
# Read in the grid data
grid = gpd.read_file("grid.shp")

# Aggregate dominance area
dissolved = grid.dissolve(by="dominant_s")
dissolved.reset_index(inplace=True)

dissolved

,dominant_s,geometry,x,y,YKR_ID,pt_r_t_Ruo,pt_r_t_Myy,pt_r_t_Iti,pt_r_t_Jum,pt_r_t_Iso,pt_r_t_Dix,pt_r_t_For,min_t
0,pt_r_t_Dixi,"MULTIPOLYGON (((384500.000 6675750.000, 384250...",392625.0,6694630.0,5810457,82.0,85.0,79.0,66.0,106.0,54.0,77.0,54.0
1,pt_r_t_Forum,"MULTIPOLYGON (((379000.000 6675250.000, 379000...",376875.0,6693130.0,5821788,119.0,106.0,124.0,109.0,131.0,125.0,105.0,105.0
2,pt_r_t_IsoOmena,"MULTIPOLYGON (((363250.000 6676750.000, 363000...",361875.0,6690130.0,5844368,82.0,73.0,106.0,107.0,72.0,93.0,80.0,72.0
3,pt_r_t_Itis,"MULTIPOLYGON (((389750.000 6672750.000, 390000...",402125.0,6685880.0,5876274,80.0,97.0,55.0,98.0,100.0,85.0,79.0,55.0
4,pt_r_t_Jumbo,"MULTIPOLYGON (((382250.000 6684000.000, 382250...",384125.0,6694630.0,5810423,134.0,107.0,142.0,104.0,159.0,105.0,126.0,104.0
5,pt_r_t_Myyrmanni,"MULTIPOLYGON (((368000.000 6681500.000, 368000...",381875.0,6697880.0,5785640,118.0,90.0,132.0,101.0,141.0,102.0,110.0,90.0
6,pt_r_t_Ruoholahti,"MULTIPOLYGON (((376250.000 6671750.000, 376000...",374125.0,6676380.0,5945730,38.0,46.0,69.0,56.0,38.0,56.0,42.0,38.0


In [4]:
# Check CRS
if dissolved.crs != pop.crs:
    dissolved = dissolved.to_crs(pop.crs)

In [5]:
# Join information between the population grid and the dominance areas
intersection = gpd.sjoin(dissolved, pop, how="inner", predicate="intersects")

In [6]:
# Report how many people live within 1.5 km distance from each shopping center
grouped = intersection.groupby("dominant_s")

for key, group in grouped:
    total_pop = group["pop18"].sum()
    print(f'{total_pop:,} people live under the dominance area of {key.split("_")[-1]}.')

219,425 people live under the dominance area of Dixi.
253,914 people live under the dominance area of Forum.
201,913 people live under the dominance area of IsoOmena.
217,581 people live under the dominance area of Itis.
76,006 people live under the dominance area of Jumbo.
233,710 people live under the dominance area of Myyrmanni.
97,493 people live under the dominance area of Ruoholahti.


Remember to print the answers in this notebook :)

### Extra bonus task

Repeat problem 2 and 3 for car accessibility! No extra points available for this, but you can for example start thinking if you want to make the final assignment related to this topic :)